In [76]:
#Duality: Fermions from Anushya's ED vs. Bosons from NLCE
#boson 1/n vs. fermion n

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

######### Plotting parameters: #########
fig_width_pt = 246.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
#fig_height = fig_width*golden_mean      # height in inches
fig_height = fig_width*0.75      # height in inches
fig_size =  [fig_width,fig_height]

rcParams['axes.labelsize']  = 10
rcParams['figure.figsize']  = fig_size
rcParams['font.family']     = 'serif'
rcParams['font.serif']      = ['Computer Modern']
rcParams['font.size']       = 10
#rcParams['text.fontsize']   = 10 ##text.fontsize is deprecated and replaced with font.size
rcParams['text.usetex']     = True
rcParams['legend.fontsize'] = 8
rcParams['xtick.labelsize'] = 8
rcParams['ytick.labelsize'] = 8

path_ferm_hi='/home/johannes/Work/boson-fermion-duality/fermions/square-in-torus/high-prec/data/Duality/'
path_ferm_std='/home/johannes/Work/boson-fermion-duality/fermions/square-in-torus/std-prec/data/Duality/'
path_bos_hi='/home/johannes/Work/boson-fermion-duality/bosons/high-prec-right-angle/data/'
path_bos_std='/home/johannes/Work/boson-fermion-duality/bosons/right-angle/data/'



from scipy.optimize import curve_fit

def FlToStr_(f):
    return str(f).replace('.','-')[:5]

def arealaw(x,b,c,d):
    return b*x + c*np.log(x) + d

C_ferm=3./(16*np.pi**2)
C_bos=3./(32*np.pi**2)


alphas_stdprec = np.linspace(1.0,5.0,41)
alphas_hiprec = 1./np.linspace(1.0,5.0,41)


sigma_inf_ferm = 0.008905/C_ferm
sigma_inf_bos = 0.005278/C_ferm

ax=plt.gca()
axins = inset_axes(ax, width="45%", height="26%", loc=4)

start=7
end=13
step=1

######### FERMIONS from direct approach ##################
c_terms=[]
for alpha in alphas_hiprec:
    alpha_s="%.3f" % alpha
    L, S = np.loadtxt(path_ferm_hi+'EE_square_M0.00_Lmult4.00_alpha'+alpha_s+'_pbc.txt',unpack=True)
    L=map(int,L)
    (b,c,d), pcov = curve_fit(arealaw, L[start:end:step], S[start:end:step])
    c_terms.append((-c/4.)/C_ferm)
    
c_terms=np.array(c_terms)
ax.plot(alphas_hiprec,alphas_hiprec**2*c_terms,label=r"$\alpha^2 \times a^f_\alpha$, $ L_{min}$="
         +str(L[start])+", $L_{max}$="+str(L[end-1]))
axins.plot(alphas_hiprec,alphas_hiprec**2*c_terms)

start=15
end=32
step=1
         
c_terms=[]
for alpha in alphas_stdprec:
    alpha_s="%.3f" % alpha
    L, S = np.loadtxt(path_ferm_std+'EE_square_M0.00_Lmult4.00_alpha'+alpha_s+'_pbc.txt',unpack=True)
    L=map(int,L)
    (b,c,d), pcov = curve_fit(arealaw, L[start:end:step], S[start:end:step])
    c_terms.append((-c/4.)/C_ferm)
    
c_terms=np.array(c_terms)
ax.plot(alphas_stdprec,alphas_stdprec**2*c_terms,label=r"$\alpha^2 \times a^f_\alpha$, $L_{min}$="
         +str(L[start])+", $L_{max}$="+str(L[end-1]))
axins.plot(alphas_stdprec,alphas_stdprec**2*c_terms)

        
#plt.plot([1,2],[1.240,4*0.7863],'.',label=r"$n^2 \times $ exact data fermions")



######### Bosons from NLCE #################
P_bos={}
start=9
end=15

a_bos=[]
for alpha in alphas_hiprec:
    order, P_bos[alpha] = np.loadtxt(path_bos_hi+'results_mass0-0_angle90_alpha'+FlToStr_(alpha)+'.txt',unpack=True)
    X=np.log(order)
    order=map(int,order)
    a_buf,b=np.polyfit(X[start:end],-P_bos[alpha][start:end],1)
    a_bos.append(a_buf)

sigma_bos=np.array(a_bos)/C_bos

ax.plot(1./alphas_hiprec,sigma_bos,label=r"$a^{cs}_{(1/\alpha)} $, $ ~~ L_{min}$="+
                  str(order[start])+", $L_{max}$="+str(order[end-1]))
axins.plot(1./alphas_hiprec,sigma_bos)

P_bos={}
start=18
end=29

a_bos=[]
for alpha in alphas_stdprec[:-10]:
    order, P_bos[alpha] = np.loadtxt(path_bos_std+'results_mass0-0_alpha'+FlToStr_(alpha)+'.txt',unpack=True)
    X=np.log(order)
    order=map(int,order)
    a_buf,b=np.polyfit(X[start:end],-P_bos[alpha][start:end],1)
    a_bos.append(a_buf)

sigma_bos=np.array(a_bos)/C_bos

ax.plot(1./alphas_stdprec[:-10],sigma_bos,label=r"$a^{cs}_{(1/\alpha)} $, $ ~~ L_{min}$="+
                  str(order[start])+", $L_{max}$="+str(order[end-1])) 
axins.plot(1./alphas_stdprec[:-10],sigma_bos)




ax.plot([2,1,1./2,1./3],[4*0.7863,1.245,0.674,0.537],'.',label="exact data")
ax.plot([0],[sigma_inf_bos],'.',label=r"$\lim_{\alpha \rightarrow \infty} a_\alpha^{cs}$")


ax.set_xlim(-0.1,5)
ax.set_ylim(0,15.5)

ax.set_xlabel(r"Renyi index - $\alpha$")
ax.set_ylabel(r"corner entanglement - $a(\pi / 2) / C_T$")

ax.legend(loc="upper left",frameon=False,numpoints=1)

plt.gcf().subplots_adjust(bottom=0.15)

axins.set_xlim(-0.1,1.1)
axins.set_ylim(0.1,1.5)

axins.set_xticks([])
axins.set_yticks([])

plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',
    right='off',
    left='off',# ticks along the top edge are off
    labelbottom='off')

#axins.plot(alphas_hiprec,alphas_hiprec**2*sigma_bos)


axins.plot([2,1,1./2,1./3],[4*0.7863,1.245,0.674,0.537],'.',label="exact data")
axins.plot([0],[sigma_inf_bos],'.',label=r"$\lim_{\alpha \rightarrow \infty} a_\alpha^{cs}$")


mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.7")


plt.savefig('ED_Fermion_NLCE_Boson_Duality1.pdf')

plt.show()

In [71]:
#Duality: Fermions from Anushya's ED vs. Bosons from NLCE
#boson n vs. fermion 1/n

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

######### Plotting parameters: #########
fig_width_pt = 246.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
#fig_height = fig_width*golden_mean      # height in inches
fig_height = fig_width*0.75      # height in inches
fig_size =  [fig_width,fig_height]

rcParams['axes.labelsize']  = 10
rcParams['figure.figsize']  = fig_size
rcParams['font.family']     = 'serif'
rcParams['font.serif']      = ['Computer Modern']
rcParams['font.size']       = 10
#rcParams['text.fontsize']   = 10 ##text.fontsize is deprecated and replaced with font.size
rcParams['text.usetex']     = True
rcParams['legend.fontsize'] = 8
rcParams['xtick.labelsize'] = 8
rcParams['ytick.labelsize'] = 8


path_ferm_hi='/home/johannes/Work/boson-fermion-duality/fermions/square-in-torus/high-prec/data/Duality/'
path_ferm_std='/home/johannes/Work/boson-fermion-duality/fermions/square-in-torus/std-prec/data/Duality/'
path_bos_hi='/home/johannes/Work/boson-fermion-duality/bosons/high-prec-right-angle/data/'
path_bos_std='/home/johannes/Work/boson-fermion-duality/bosons/right-angle/data/'


from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

def FlToStr_(f):
    return str(f).replace('.','-')[:5]

def arealaw(x,b,c,d):
    return b*x + c*np.log(x) + d

C_ferm=3./(16*np.pi**2)
C_bos=3./(32*np.pi**2)

sigma_inf_ferm = 0.008905/C_ferm
sigma_inf_bos = 0.005278/C_ferm


alphas_stdprec = np.linspace(1.0,5.0,41)
alphas_hiprec = 1./np.linspace(1.0,5.0,41)

ax=plt.gca()
axins = inset_axes(ax, width="45%", height="27%", loc=4)


start=7
end=13
step=1

######### FERMIONS from direct approach ##################
c_terms=[]
for alpha in alphas_hiprec:
    alpha_s="%.3f" % alpha
    L, S = np.loadtxt(path_ferm_hi+'EE_square_M0.00_Lmult4.00_alpha'+alpha_s+'_pbc.txt',unpack=True)
    L=map(int,L)
    (b,c,d), pcov = curve_fit(arealaw, L[start:end:step], S[start:end:step])
    c_terms.append((-c/4.)/C_ferm)
    
c_terms=np.array(c_terms)
ax.plot(1./alphas_hiprec,c_terms,label=r"$a^f_{(1/\alpha)}$, $L_{min}$="
         +str(L[start])+", $L_{max}$="+str(L[end-1]))
axins.plot(1./alphas_hiprec,c_terms)

start=15
end=32
step=1
         
c_terms=[]
for alpha in alphas_stdprec:
    alpha_s="%.3f" % alpha
    L, S = np.loadtxt(path_ferm_std+'EE_square_M0.00_Lmult4.00_alpha'+alpha_s+'_pbc.txt',unpack=True)
    L=map(int,L)
    (b,c,d), pcov = curve_fit(arealaw, L[start:end:step], S[start:end:step])
    c_terms.append((-c/4.)/C_ferm)
    
c_terms=np.array(c_terms)
ax.plot(1./alphas_stdprec,c_terms,label=r"$a^f_{(1/\alpha)}$, $L_{min}$="
         +str(L[start])+", $L_{max}$="+str(L[end-1]))
axins.plot(1./alphas_stdprec,c_terms)

#plt.plot([1,2],[1.240,4*0.7863],'.',label=r"$n^2 \times $ exact data fermions")



######### Bosons from NLCE #################
P_bos={}
start=9
end=15

a_bos=[]
for alpha in alphas_hiprec:
    order, P_bos[alpha] = np.loadtxt(path_bos_hi+'results_mass0-0_angle90_alpha'+FlToStr_(alpha)+'.txt',unpack=True)
    X=np.log(order)
    order=map(int,order)
    a_buf,b=np.polyfit(X[start:end],-P_bos[alpha][start:end],1)
    a_bos.append(a_buf)

sigma_bos=np.array(a_bos)/C_bos

ax.plot(alphas_hiprec,alphas_hiprec**2*sigma_bos,label=r"$\alpha^2 \times a^{cs}_{\alpha}$, $L_{min}$="+
                  str(order[start])+", $L_{max}$="+str(order[end-1])) 
axins.plot(alphas_hiprec,alphas_hiprec**2*sigma_bos)


P_bos={}
start=18
end=29

a_bos=[]
for alpha in alphas_stdprec[:-10]:
    order, P_bos[alpha] = np.loadtxt(path_bos_std+'results_mass0-0_alpha'+FlToStr_(alpha)+'.txt',unpack=True)
    X=np.log(order)
    order=map(int,order)
    a_buf,b=np.polyfit(X[start:end],-P_bos[alpha][start:end],1)
    a_bos.append(a_buf)

sigma_bos=np.array(a_bos)/C_bos

ax.plot(alphas_stdprec[:-10],alphas_stdprec[:-10]**2*sigma_bos,label=r"$\alpha^2 \times a^{cs}_{\alpha}$, $L_{min}$="+
                  str(order[start])+", $L_{max}$="+str(order[end-1])) 
axins.plot(alphas_stdprec[:-10],alphas_stdprec[:-10]**2*sigma_bos)




ax.plot([1./2,1,2,3],[0.7863,1.245,4*0.674,9*0.537],'.',label="exact data")
ax.plot([0],[sigma_inf_ferm],'.',label=r"$\lim_{\alpha \rightarrow \infty} a_\alpha^{f}$")


ax.set_xlim(-0.1,5)
ax.set_ylim(0,11.5)

ax.set_xlabel(r"Renyi index - $\alpha$")
ax.set_ylabel(r"corner entanglement - $a(\pi / 2)/C_T$")

ax.legend(loc="upper left",frameon=False,numpoints=1)

plt.gcf().subplots_adjust(bottom=0.15)

### Inset #####

axins.set_xlim(-0.1,1.1)
axins.set_ylim(0.1,1.5)

axins.set_xticks([])
axins.set_yticks([])

plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',
    right='off',
    left='off',# ticks along the top edge are off
    labelbottom='off')

#axins.plot(alphas_hiprec,alphas_hiprec**2*sigma_bos)


axins.plot([1./2,1,2,3],[0.7863,1.245,4*0.674,9*0.537],'.',label="exact data")
axins.plot([0],[sigma_inf_ferm],'.',label=r"$\lim_{\alpha \rightarrow \infty} a_\alpha^{f}$")

mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.7")


plt.savefig('ED_Fermion_NLCE_Boson_Duality2.pdf')

plt.show()